In [1]:
# Libraries for working with data
import numpy as np 
import pandas as pd

# libraries for visualizing data
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


#########################################################
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [2]:
#Loading dataset
train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26298 entries, 0 to 26297
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   row_id    26298 non-null  int64 
 1   date      26298 non-null  object
 2   country   26298 non-null  object
 3   store     26298 non-null  object
 4   product   26298 non-null  object
 5   num_sold  26298 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.2+ MB


In [4]:
# converting date into time series data
train['date'] = pd.to_datetime(train.date)
test['date'] = pd.to_datetime(test.date)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26298 entries, 0 to 26297
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   row_id    26298 non-null  int64         
 1   date      26298 non-null  datetime64[ns]
 2   country   26298 non-null  object        
 3   store     26298 non-null  object        
 4   product   26298 non-null  object        
 5   num_sold  26298 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.2+ MB


In [6]:
# duration 
print(f'Training data is from {train.date.min()} to {train.date.max()} duration {(train.date.max()-train.date.min())} ')
print(f'Testing  data is from {test.date.min()} to {test.date.max()} duration  {test.date.max()-test.date.min()} ')

Training data is from 2015-01-01 00:00:00 to 2018-12-31 00:00:00 duration 1460 days 00:00:00 
Testing  data is from 2019-01-01 00:00:00 to 2019-12-31 00:00:00 duration  364 days 00:00:00 


In [7]:
for col in ['country', 'store','product']:
    print(f'{col}: {train[col].unique()}')


country: ['Finland' 'Norway' 'Sweden']
store: ['KaggleMart' 'KaggleRama']
product: ['Kaggle Mug' 'Kaggle Hat' 'Kaggle Sticker']


In [8]:
train_categories = train.groupby(['country','store','product']).agg({'product': 'count'})
train_categories

product
country store      product                
Finland KaggleMart Kaggle Hat         1461
                   Kaggle Mug         1461
                   Kaggle Sticker     1461
        KaggleRama Kaggle Hat         1461
                   Kaggle Mug         1461
                   Kaggle Sticker     1461
Norway  KaggleMart Kaggle Hat         1461
                   Kaggle Mug         1461
                   Kaggle Sticker     1461
        KaggleRama Kaggle Hat         1461
                   Kaggle Mug         1461
                   Kaggle Sticker     1461
Sweden  KaggleMart Kaggle Hat         1461
                   Kaggle Mug         1461
                   Kaggle Sticker     1461
        KaggleRama Kaggle Hat         1461
                   Kaggle Mug         1461
                   Kaggle Sticker     1461

In [9]:
test_categories = test.groupby(['country','store','product']).agg({'product': 'count'})
test_categories

product
country store      product                
Finland KaggleMart Kaggle Hat          365
                   Kaggle Mug          365
                   Kaggle Sticker      365
        KaggleRama Kaggle Hat          365
                   Kaggle Mug          365
                   Kaggle Sticker      365
Norway  KaggleMart Kaggle Hat          365
                   Kaggle Mug          365
                   Kaggle Sticker      365
        KaggleRama Kaggle Hat          365
                   Kaggle Mug          365
                   Kaggle Sticker      365
Sweden  KaggleMart Kaggle Hat          365
                   Kaggle Mug          365
                   Kaggle Sticker      365
        KaggleRama Kaggle Hat          365
                   Kaggle Mug          365
                   Kaggle Sticker      365

In [10]:
train_month = train.set_index('date').groupby([pd.Grouper(freq='M'), 'country', 'store', 'product'])['num_sold'].mean().unstack([1, 2, 3]).reset_index()
train_month_country = train.set_index('date').groupby([pd.Grouper(freq='M'), 'country'])['num_sold'].mean().unstack([1]).reset_index()
train_month_country.head()


country,date,Finland,Norway,Sweden
0,2015-01-31,311.817204,529.779570,378.064516
1,2015-02-28,295.702381,511.892857,356.505952
2,2015-03-31,300.725806,521.440860,362.327957
3,2015-04-30,313.016667,558.916667,375.722222
4,2015-05-31,303.483871,551.612903,365.096774


In [11]:
train_date = train.set_index('date').pivot(columns=['country','store','product'], values='num_sold').reset_index()
train_date.head()

country       date    Finland                                                  \
store              KaggleMart                           KaggleRama              
product            Kaggle Mug Kaggle Hat Kaggle Sticker Kaggle Mug Kaggle Hat   
0       2015-01-01        329        520            146        572        911   
1       2015-01-02        318        493            127        544        794   
2       2015-01-03        360        535            141        579        843   
3       2015-01-04        332        544            125        582        873   
4       2015-01-05        243        378            102        423        668   

country                    Norway                                       \
store                  KaggleMart                           KaggleRama   
product Kaggle Sticker Kaggle Mug Kaggle Hat Kaggle Sticker Kaggle Mug   
0                  283        526        906            250       1005   
1                  249        538        845            237        949   
2                  265        614        930            254       1029   
3                  245        589        889            255        957   
4                  187        435        671            184        739   

country                               Sweden                            \
store                             KaggleMart                             
product Kaggle Hat Kaggle Sticker Kaggle Mug Kaggle Hat Kaggle Sticker   
0             1461            395        440        624            175   
1             1446            415        384        572            155   
2             1631            451        420        630            173   
3             1540            455        406        697            169   
4             1126            310        298        442            127   

country                                       
store   KaggleRama                            
product Kaggle Mug Kaggle Hat Kaggle Sticker  
0              706       1195            324  
1              653        986            293  
2              734       1167            319  
3              657       1112            300  
4              512        796            227

In [12]:
test.head()

,row_id,date,country,store,product
0,26298,2019-01-01,Finland,KaggleMart,Kaggle Mug
1,26299,2019-01-01,Finland,KaggleMart,Kaggle Hat
2,26300,2019-01-01,Finland,KaggleMart,Kaggle Sticker
3,26301,2019-01-01,Finland,KaggleRama,Kaggle Mug
4,26302,2019-01-01,Finland,KaggleRama,Kaggle Hat


In [13]:
test_data =test.pivot(index='date',columns=['country','store','product'])
test_data

row_id                                                  \
country       Finland                                                   
store      KaggleMart                           KaggleRama              
product    Kaggle Mug Kaggle Hat Kaggle Sticker Kaggle Mug Kaggle Hat   
date                                                                    
2019-01-01      26298      26299          26300      26301      26302   
2019-01-02      26316      26317          26318      26319      26320   
2019-01-03      26334      26335          26336      26337      26338   
2019-01-04      26352      26353          26354      26355      26356   
2019-01-05      26370      26371          26372      26373      26374   
...               ...        ...            ...        ...        ...   
2019-12-27      32778      32779          32780      32781      32782   
2019-12-28      32796      32797          32798      32799      32800   
2019-12-29      32814      32815          32816      32817      32818   
2019-12-30      32832      32833          32834      32835      32836   
2019-12-31      32850      32851          32852      32853      32854   

                                                                           \
country                       Norway                                        
store                     KaggleMart                           KaggleRama   
product    Kaggle Sticker Kaggle Mug Kaggle Hat Kaggle Sticker Kaggle Mug   
date                                                                        
2019-01-01          26303      26304      26305          26306      26307   
2019-01-02          26321      26322      26323          26324      26325   
2019-01-03          26339      26340      26341          26342      26343   
2019-01-04          26357      26358      26359          26360      26361   
2019-01-05          26375      26376      26377          26378      26379   
...                   ...        ...        ...            ...        ...   
2019-12-27          32783      32784      32785          32786      32787   
2019-12-28          32801      32802      32803          32804      32805   
2019-12-29          32819      32820      32821          32822      32823   
2019-12-30          32837      32838      32839          32840      32841   
2019-12-31          32855      32856      32857          32858      32859   

                                                                           \
country                                  Sweden                             
store                                KaggleMart                             
product    Kaggle Hat Kaggle Sticker Kaggle Mug Kaggle Hat Kaggle Sticker   
date                                                                        
2019-01-01      26308          26309      26310      26311          26312   
2019-01-02      26326          26327      26328      26329          26330   
2019-01-03      26344          26345      26346      26347          26348   
2019-01-04      26362          26363      26364      26365          26366   
2019-01-05      26380          26381      26382      26383          26384   
...               ...            ...        ...        ...            ...   
2019-12-27      32788          32789      32790      32791          32792   
2019-12-28      32806          32807      32808      32809          32810   
2019-12-29      32824          32825      32826      32827          32828   
2019-12-30      32842          32843      32844      32845          32846   
2019-12-31      32860          32861      32862      32863          32864   

                                                 
country                                          
store      KaggleRama                            
product    Kaggle Mug Kaggle Hat Kaggle Sticker  
date                                             
2019-01-01      26313      26314          26315  
2019-01-02      26331      26332          26333  
2019-01-03      26349      263

In [14]:
test_date = test.set_index('date').pivot(columns=['country','store','product']).reset_index()
test_date.head()

date     row_id                                                  \
country               Finland                                                   
store              KaggleMart                           KaggleRama              
product            Kaggle Mug Kaggle Hat Kaggle Sticker Kaggle Mug Kaggle Hat   
0       2019-01-01      26298      26299          26300      26301      26302   
1       2019-01-02      26316      26317          26318      26319      26320   
2       2019-01-03      26334      26335          26336      26337      26338   
3       2019-01-04      26352      26353          26354      26355      26356   
4       2019-01-05      26370      26371          26372      26373      26374   

                                                                        \
country                    Norway                                        
store                  KaggleMart                           KaggleRama   
product Kaggle Sticker Kaggle Mug Kaggle Hat Kaggle Sticker Kaggle Mug   
0                26303      26304      26305          26306      26307   
1                26321      26322      26323          26324      26325   
2                26339      26340      26341          26342      26343   
3                26357      26358      26359          26360      26361   
4                26375      26376      26377          26378      26379   

                                                                        \
country                               Sweden                             
store                             KaggleMart                             
product Kaggle Hat Kaggle Sticker Kaggle Mug Kaggle Hat Kaggle Sticker   
0            26308          26309      26310      26311          26312   
1            26326          26327      26328      26329          26330   
2            26344          26345      26346      26347          26348   
3            26362          26363      26364      26365          26366   
4            26380          26381      26382      26383          26384   

                                              
country                                       
store   KaggleRama                            
product Kaggle Mug Kaggle Hat Kaggle Sticker  
0            26313      26314          26315  
1            26331      26332          26333  
2            26349      26350          26351  
3            26367      26368          26369  
4            26385      26386          26387

In [15]:
train_data_by_year = train.set_index('date').groupby([pd.Grouper(freq='Y')])['num_sold'].mean().reset_index()
train_data_by_year.head()

,date,num_sold
0,2015-12-31,364.382192
1,2016-12-31,364.807529
2,2017-12-31,396.011720
3,2018-12-31,424.995129


In [16]:
train_year = train.set_index('date').groupby([pd.Grouper(freq='M')])[['num_sold']].mean().reset_index()
train_year.head()

,date,num_sold
0,2015-01-31,406.553763
1,2015-02-28,388.033730
2,2015-03-31,394.831541
3,2015-04-30,415.885185
4,2015-05-31,406.731183


In [17]:
fig_4 = px.line(train_year,x='date',y='num_sold')
fig_4.show()

## Preprocessing and visualization 